In [2]:
# Imports
import mido as m

# Files
all_star_real = m.MidiFile('C:\\Users\Tim\Documents\GitHub\purdue-AIMA-basic-pitch-testing\Tim\'s songs\All Star, but a Barbershop Choir (Stress Test)\\all star choir.mid')
all_star_not_real = m.MidiFile('C:\\Users\Tim\Documents\GitHub\purdue-AIMA-basic-pitch-testing\Tim\'s songs\All Star, but a Barbershop Choir (Stress Test)\\all star choir E-PIANO_basic_pitch.mid')
rush_e_real = m.MidiFile("C:\\Users\Tim\Documents\GitHub\purdue-AIMA-basic-pitch-testing\Tim\'s songs\Rush E (but jazz)\\rush_e.mid")
rush_e_not_real = m.MidiFile("C:\\Users\Tim\Documents\GitHub\purdue-AIMA-basic-pitch-testing\Tim\'s songs\Rush E (but jazz)\\rush_e E_PIANO_basic_pitch.mid")
bcr_orig = m.MidiFile("C:\\Users\\Tim\\Documents\\GitHub\\purdue-AIMA-basic-pitch-testing\\Tim\'s songs\\Blind Composition Render\\blind comp render E-PIANO ONLY_basic_pitch.mid")
bcr_clean = m.MidiFile("C:\\Users\\Tim\\Documents\\GitHub\\purdue-AIMA-basic-pitch-testing\\Tim\'s songs\\Blind Composition Render\\Further Experimentation\\blind comp render PREDICTED clean.mid")
bcr_medium = m.MidiFile("C:\\Users\\Tim\\Documents\\GitHub\\purdue-AIMA-basic-pitch-testing\\Tim\'s songs\\Blind Composition Render\\Further Experimentation\\blind comp render PREDICTED more notes.mid")
bcr_all = m.MidiFile("C:\\Users\\Tim\\Documents\\GitHub\\purdue-AIMA-basic-pitch-testing\\Tim\'s songs\\Blind Composition Render\\Further Experimentation\\blind comp render PREDICTED all the notes.mid")

In [3]:
# Environment variables
n_shingle = 3
round_place = 1

In [4]:
# Helper methods
def printNoteName(integer):
    out = "";
    integer = int(integer)
    if integer % 12 == 0:
        out += "C"
    elif integer % 12 == 1:
        out += "C#"
    elif integer % 12 == 2:
        out += "D"
    elif integer % 12 == 3:
        out += "D#"
    elif integer % 12 == 4:
        out += "E"
    elif integer % 12 == 5:
        out += "F"
    elif integer % 12 == 6:
        out += "F#"
    elif integer % 12 == 7:
        out += "G"
    elif integer % 12 == 8:
        out += "G#"
    elif integer % 12 == 9:
        out += "A"
    elif integer % 12 == 10:
        out += "A#"
    elif integer % 12 == 11:
        out += "B"
    out += str(int(integer / 12 - 1));
    return out

# Returns the intersection of two lists
def intersection(A, B):
    interSet = []
    for a in A:
        if a in B and not a in interSet:
            interSet.append(a)
    return interSet

# Returns the union of two lists
def union(A, B):
    unionSet = []
    for a in A:
        unionSet.append(a)
    for b in B:
        if not (b in unionSet):
            unionSet.append(b)
    return unionSet

In [5]:
## Single pitch binning

# Returns a list of pitch-agnostic start times 
def startTimes(midi, round_place):
    # Grab list of midi messages
    midimsgs = []
    currentTime = 0.0
    modifiedTimes = []
    for i in midi:
        if i.type == 'note_on' or i.type == 'note_off':
            modifiedTimes.append(currentTime + i.time)
            currentTime += i.time
    # Adjust first note start time to be 0
    for i in range(1, len(modifiedTimes)):
        modifiedTimes[i] -= modifiedTimes[0]
        modifiedTimes[i] = round(modifiedTimes[i], round_place)
    if len(modifiedTimes) != 0: modifiedTimes[0] = 0.0
    # Remove duplicates
    noDuplicatesList = []
    for item in modifiedTimes:
        if not item in noDuplicatesList:
            noDuplicatesList.append(item)
    return noDuplicatesList

# Returns a list of shingles of elements in a 1D list
def nShingle(myList, n):
    if n > len(myList): return []
    if n == len(myList): return myList
    shingleList = []
    for i in range(0, len(myList) - n + 1):
        shingle = []
        for j in range(i, i + n):
            shingle.append(myList[j])
        shingleList.append(shingle)
    return shingleList

# Returns a pitch-agnostic similarity coefficient of two MIDI files
def similarity(midi1, midi2, n_shingle, round_place):
    startTimes1 = startTimes(midi1, round_place)
    startTimes2 = startTimes(midi2, round_place)
    shingles1 = nShingle(startTimes1, n_shingle)
    shingles2 = nShingle(startTimes2, n_shingle)
    return (float)(len(intersection(shingles1, shingles2))) / (float)(len(union(shingles1, shingles2))) * 100

In [6]:
## n-pitch binning

# Returns a list of pitch-separated start times 
def startTimesPitched(midi, round_place, octave_ignored):
    number_notes = 128
    if (octave_ignored):
        number_notes = 12
    first_note_start_time = -1.0
    note_startTimes = [None] * number_notes
    for i in range(0, number_notes):
        note_startTimes[i] = []
    currentTime = 0.0
    triggered = False
    # Grab list of notes sorted by pitch (either octave-agnostic or dependent)
    for i in midi:
        triggered = True
        if i.type == 'note_on' or i.type == 'note_off':
            if (octave_ignored):
                note_startTimes[i.note % 12].append(currentTime + i.time)
            else:
                note_startTimes[i.note].append(currentTime + i.time)
            if (first_note_start_time == -1.0): first_note_start_time = i.time
            currentTime += i.time
    # Adjust start time to be 0
    if triggered:
        for i in range(0, len(note_startTimes)):
            for j in range(0, len(note_startTimes[i])):
                note_startTimes[i][j] -= first_note_start_time
                note_startTimes[i][j] = round(note_startTimes[i][j], round_place)
    # Remove duplicates
    for i in range(0, len(note_startTimes)):
        noDuplicatesList = []
        for j in range(0, len(note_startTimes[i])):
            if not note_startTimes[i][j] in noDuplicatesList:
                noDuplicatesList.append(note_startTimes[i][j])
        note_startTimes[i] = noDuplicatesList
    return note_startTimes

# Returns a list of shingles of a 2D list
def nShinglePitched(my2Dlist, n):
    fullList = [None] * len(my2Dlist)
    for i in range(0, len(my2Dlist)):
        fullList[i] = nShingle(my2Dlist[i], n)
    return fullList

# Returns a pitch-adjusted similarity coefficient of two MIDI files
def similarityPitched(midi1, midi2, n_shingle, round_place, octave_ignored):
    startTimesPitched1 = startTimesPitched(midi1, round_place, octave_ignored)
    startTimesPitched2 = startTimesPitched(midi2, round_place, octave_ignored)
    shinglesPitched1 = nShinglePitched(startTimesPitched1, n_shingle)
    shinglesPitched2 = nShinglePitched(startTimesPitched2, n_shingle)
    weighted_sum = 0.0
    total_shingles = 0.0
    for i in range(0, len(startTimesPitched1)):
        sim_denominator = (float)(len(union(shinglesPitched1[i], shinglesPitched2[i])))
        if not sim_denominator == 0:
            sim_numerator = (float)(len(intersection(shinglesPitched1[i], shinglesPitched2[i])))
            similarity = sim_numerator / sim_denominator
            shingles_added = (len(shinglesPitched1[i]) + len(shinglesPitched2[i]))
            weighted_sum += similarity * shingles_added
            total_shingles += shingles_added
    score = weighted_sum / total_shingles * 100
    return score

In [7]:
# Same file comparison
print(similarityPitched(all_star_real, all_star_real, n_shingle, round_place, False))
print(similarityPitched(rush_e_real, rush_e_real, n_shingle, round_place, False))

100.0
100.0


In [8]:
print("Similarities, the original output E_piano with the clean, medium, and crazy amount of notes, respectively")
print()
print("All notes treated equally:")
print(similarity(bcr_orig, bcr_clean, n_shingle, round_place))
print(similarity(bcr_orig, bcr_medium, n_shingle, round_place))
print(similarity(bcr_orig, bcr_all, n_shingle, round_place))
print()
print("All notes treated separately:")
print(similarityPitched(bcr_orig, bcr_clean, n_shingle, round_place, False))
print(similarityPitched(bcr_orig, bcr_medium, n_shingle, round_place, False))
print(similarityPitched(bcr_orig, bcr_all, n_shingle, round_place, False))
print()
print("All notes treated as in one octave:")
print(similarityPitched(bcr_orig, bcr_clean, n_shingle, round_place, True))
print(similarityPitched(bcr_orig, bcr_medium, n_shingle, round_place, True))
print(similarityPitched(bcr_orig, bcr_all, n_shingle, round_place, True))

Similarities, the original output E_piano with the clean, medium, and crazy amount of notes, respectively

All notes treated equally:
15.517241379310345
6.3444108761329305
31.655290102389078

All notes treated separately:
0.0
0.0
0.2602008381424062

All notes treated as in one octave:
0.34568246293483873
0.4903821392771805
2.3259256647659936
